In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [91]:
from sklearn.model_selection import train_test_split

dataset_path = './dataset'
target_column = 'target_reg'

train_df = pd.read_csv(f'{dataset_path}/train.csv')
train_df.dropna(inplace=True)
print(train_df.isnull().sum().sum())


0


In [92]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

columns_to_onehotencode = [
    'MEGYE_2014',
    'MEGYE_2015', 
    'MEGYE_2016', 
    'SZEKHELY_IR_SZAM_2014', 
    'SZEKHELY_IR_SZAM_2015', 
    'SZEKHELY_IR_SZAM_2016', 
    'GAZDALKODASI_FORMA_2014_fo_kategoria',
	'GAZDALKODASI_FORMA_2014_1_alkategoria',	
    'GAZDALKODASI_FORMA_2014_2_alkategoria',	
    'GAZDALKODASI_FORMA_2015_fo_kategoria',	
    'GAZDALKODASI_FORMA_2015_1_alkategoria',	
    'GAZDALKODASI_FORMA_2015_2_alkategoria',	
    'GAZDALKODASI_FORMA_2016_fo_kategoria',	
    'GAZDALKODASI_FORMA_2016_1_alkategoria',	
    'GAZDALKODASI_FORMA_2016_2_alkategoria',
    'KKV_BESOROLAS_2014_fo_kategoria',
    'KKV_BESOROLAS_2014_1_alkategoria',
    'KKV_BESOROLAS_2014_2_alkategoria',
    'KKV_BESOROLAS_2015_fo_kategoria',
    'KKV_BESOROLAS_2015_1_alkategoria',
    'KKV_BESOROLAS_2015_2_alkategoria',
    'KKV_BESOROLAS_2016_fo_kategoria',
    'KKV_BESOROLAS_2016_1_alkategoria',
    'KKV_BESOROLAS_2016_2_alkategoria',
    'agazat_2014_fo_kategoria',
    'agazat_2014_1_alkategoria',
    'agazat_2014_2_alkategoria',
    'agazat_2014_3_alkategoria',
    'agazat_2015_fo_kategoria',
    'agazat_2015_1_alkategoria',
    'agazat_2015_2_alkategoria',
    'agazat_2015_3_alkategoria',
    'agazat_2016_fo_kategoria',
    'agazat_2016_1_alkategoria',
    'agazat_2016_2_alkategoria',
    'agazat_2016_3_alkategoria'
]

encoder_df = pd.DataFrame(encoder.fit_transform(train_df[[*columns_to_onehotencode]]).toarray())

train_df = train_df.join(encoder_df)
train_df.iloc[:1000,:].to_csv('train_df.csv')
print(train_df.shape)
print(train_df.isnull().sum().sum())

(31745, 6702)
8764380


In [93]:
target_df = train_df[target_column]
train_df.drop([target_column, *columns_to_onehotencode], axis='columns', inplace=True)

train_df.columns = train_df.columns.map(str)

x_train, x_test, y_train, y_test = train_test_split(train_df, target_df, random_state=42)

In [94]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

predictions = lin_reg.predict(x_test)

mse = mean_squared_error(predictions, y_test)
print(f'mean squared error: {mse}')

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values